<a href="https://colab.research.google.com/github/arturomolin/Tarea1/blob/main/Ejercicio25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Ejercicio 25
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import norm, cholesky, qr, solve

# Definimos los parámetros
m = 21  # número de puntos de datos
n = 12  # grado del polinomio + 1 (grado n-1)
epsilon = 1e-10  # magnitud de la perturbación

# Vamos a genarar los puntos t_i uniformemente espaciados en [0,1]
t_i = np.linspace(0, 1, m)

# Definimos la matriz de Vandermonde para el ajuste polinomial
A = np.vander(t_i, n, increasing=True)

# Damos los valores "exactos" y_i (sin perturbación)
x_exact = np.ones(n)  # coeficientes exactos (todos 1)
y_exact = A @ x_exact

# Definimos la perturbación aleatoria
np.random.seed(42)  # Para reproducibilidad
u_i = np.random.rand(m)
perturbacion = (2 * u_i - 1) * epsilon
y_i = y_exact + perturbacion

# Método 1: Ecuaciones Normales con Factorización de Cholesky

# Formamos el sistema normal A^T A x = A^T y
ATA = A.T @ A
ATy = A.T @ y_i

try:
    # Hacemos la factorización de Cholesky
    L = cholesky(ATA)
    # Resolvemos para L L^T x = A^T y en dos pasos
    y = solve(L, ATy)
    x_cholesky = solve(L.T, y)
except np.linalg.LinAlgError:
    print("La matriz A^T A no es definida positiva para Cholesky")
    x_cholesky = np.linalg.solve(ATA, ATy)  # Usamos solver general si Cholesky falla

# Método 2: Factorización QR

Q, R = qr(A, mode='reduced')  # Factorizamos QR reducida
x_qr = solve(R, Q.T @ y_i)

# Calculamos los errores
error_cholesky = norm(x_cholesky - x_exact)
error_qr = norm(x_qr - x_exact)

# Evaluaamos los polinomios ajustados en puntos t_i
y_cholesky = A @ x_cholesky
y_qr = A @ x_qr

# Resultados
print("Coeficientes exactos:", x_exact)
print("\nCoeficientes por Ecuaciones Normales (Cholesky):", x_cholesky)
print("Error:", error_cholesky)
print("\nCoeficientes por Factorización QR:", x_qr)
print("Error:", error_qr)

# Graficamos para ver los ajustes
plt.figure(figsize=(12, 6))
plt.plot(t_i, y_exact, 'ko', label='Datos exactos')
plt.plot(t_i, y_i, 'ro', label='Datos perturbados')
plt.plot(t_i, y_cholesky, 'b-', label='Ajuste Cholesky')
plt.plot(t_i, y_qr, 'g--', label='Ajuste QR')
plt.legend()
plt.title('Comparación de métodos de ajuste polinomial')
plt.xlabel('t_i')
plt.ylabel('y_i')
plt.grid(True)
plt.show()

# PREGUNTAS

a) El método de Ecuaciones Normales es más sensible a la perturbación porque:



*   Forma la matriz AᵀA, que cuadra el número de condición de la matriz original A
*  Los errores de redondeo se amplifican

*   La factorización QR trabaja directamente con la matriz A sin formar AᵀA

Por lo tanto, el error en la solución de Cholesky es significativamente mayor que el de QR.

b) La proximidad a la solución exacta de la factorización QR proporciona una solución más cercana a los valores exactos porque:

* Es más estable para problemas mal condicionados

* Evita la formación explícita de AᵀA que amplifica los errores

* Los errores numéricos se propagan menos en el proceso de solución

Por lo tanto, el error de QR será de magnitud menor que el de Cholesky.

c) La diferencia en el ajuste de los puntos (t_i, y_i) puede ser pequeña, ya que un problema mal condicionado tiene muchas soluciones de coeficientes que producen ajustes similares y los polinomios de alto grado pueden ser muy sensibles a cambios en coeficientes pero producir valores similares en los puntos de datos. Como se muestra en la grafica, ambas curvas de ajuste pasan muy cerca de los puntos perturbados, a pesar de la diferencia de los coeficientes.